In [1]:
from langchain.llms import GooglePalm

api_key = 'your api key' # get this free api key from https://makersuite.google.com/

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [2]:
poem = llm("Write a 4 line poem of my love for  parents")
print(poem)

**Love for My Parents**

I love my parents,
They're always there for me.
They're my biggest fans,
And I'm so lucky to have them.


In [3]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

Now let's load data from E-commerece response csv file

In [4]:
from langchain.document_loaders.csv_loader import CSVLoader

# Corrected file path
file_path = 'C:/Users/shubham rana/Downloads/data.csv'

# Creating an instance of CSVLoader
loader = CSVLoader(file_path=file_path, source_column="prompt")

# Store the loaded data in the 'data' variable
data = loader.load()


In [7]:
data

[Document(page_content='prompt: \'How can I create an account?\'\nresponse: "To create an account click on the \'Sign Up\' button on the top right corner of our website and follow the instructions to complete the registration process."}', metadata={'source': " 'How can I create an account?'", 'row': 0}),
 Document(page_content="prompt: 'What payment methods do you accept?'\nresponse: 'We accept major credit cards debit cards and PayPal as payment methods for online orders.'}", metadata={'source': " 'What payment methods do you accept?'", 'row': 1}),
 Document(page_content='prompt: \'How can I track my order?\'\nresponse: "You can track your order by logging into your account and navigating to the \'Order History\' section. There you will find the tracking information for your shipment."}', metadata={'source': " 'How can I track my order?'", 'row': 2}),
 Document(page_content="prompt: 'What is your return policy?'\nresponse: 'Our return policy allows you to return products within 30 day

Hugging Face Embeddings

In [8]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("Can I return a product if I changed my mind?")


load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
e[:5]

[-0.043898072093725204,
 0.00768554350361228,
 -0.009231901727616787,
 0.02449624426662922,
 0.03359228000044823]

In [9]:
len(e)

768

As you can see above, embedding for a sentance "Can I return a product if I changed my mind" is a list of size 768. Looking at the numbers in this list, doesn't give any intuitive understanding of what it is but just assume that these numbers are capturing the meaning of "What is your refund policy". If you are curious to know about embeddings, go to youtube and search "codebasics word embeddings" and you will find bunch of videos with simple, intuitive explanations

Vector store using FAISS

In [10]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [11]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content="prompt: 'How can I contact customer support?'\nresponse: 'You can contact our customer support team by phone at [phone number] or by email at [email address]. Our team is available [working hours] to assist you with any inquiries or issues you may have.'}", metadata={'source': " 'How can I contact customer support?'", 'row': 9}),
 Document(page_content="prompt: 'Do you offer installation services for your products?'\nresponse: 'Installation services are available for select products. Please check the product description or contact our customer support team for more information and to request installation services.'}", metadata={'source': " 'Do you offer installation services for your products?'", 'row': 30}),
 Document(page_content='prompt: "Can I order a product if it is listed as \'out of stock\' but available for pre-order?"\nresponse: \'If a product is available for pre-order you can place an order to secure your item. The product will be shipped once it bec

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

Create RetrievalQA chain along with prompt template 🚀

In [12]:

from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [13]:
chain('I return a product if it was purchased with a promotional gift card?')

{'query': 'I return a product if it was purchased with a promotional gift card?',
 'result': 'Yes you can return a product purchased with a promotional gift card. The refund will be issued in the form of store credit or a new gift card.',
 'source_documents': [Document(page_content="prompt: 'Can I return a product if it was purchased with a promotional gift card?'\nresponse: 'Yes you can return a product purchased with a promotional gift card. The refund will be issued in the form of store credit or a new gift card.'}", metadata={'source': " 'Can I return a product if it was purchased with a promotional gift card?'", 'row': 75}),
  Document(page_content="prompt: 'Can I return a product if it was purchased with a gift card?'\nresponse: 'Yes you can return a product purchased with a gift card. The refund will be issued in the form of store credit or a new gift card.'}", metadata={'source': " 'Can I return a product if it was purchased with a gift card?'", 'row': 51}),
  Document(page_con

In [14]:
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: \'How can I create an account?\'\nresponse: "To create an account click on the \'Sign Up\' button on the top right corner of our website and follow the instructions to complete the registration process."}', metadata={'source': " 'How can I create an account?'", 'row': 0}),
  Document(page_content="prompt: 'Do you have a loyalty program?'\nresponse: 'Yes we have a loyalty program where you can earn points for every purchase. These points can be redeemed for discounts on future orders. Please visit our website to learn more and join the program.'}", metadata={'source': " 'Do you have a loyalty program?'", 'row': 15}),
  Document(page_content="prompt: 'Can I order a product that is out of stock?'\nresponse: 'If a product is currently out of stock you will usually see an option to sign up for product notifications. This way you will be alerted when the product become

In [16]:
chain("Can I order a product if it is listed as backordered?")

{'query': 'Can I order a product if it is listed as backordered?',
 'result': "Products listed as 'backordered' are temporarily out of stock but can still be ordered. Your order will be fulfilled once the product is restocked.",
 'source_documents': [Document(page_content='prompt: "Can I order a product if it is listed as \'backordered\'?"\nresponse: "Products listed as \'backordered\' are temporarily out of stock but can still be ordered. Your order will be fulfilled once the product is restocked."}', metadata={'source': ' "Can I order a product if it is listed as \'backordered\'?"', 'row': 44}),
  Document(page_content='prompt: "Can I order a product if it is listed as \'out of stock\' but available for backorder?"\nresponse: "If a product is listed as \'out of stock\' but available for backorder you can place an order to secure your item. The product will be shipped once it becomes available."}', metadata={'source': ' "Can I order a product if it is listed as \'out of stock\' but av